In [2]:
import pandas as pd
import zipfile
import seaborn as sns # For plotting
import matplotlib.pyplot as plt # For showing plots
import numpy as np
import pandas as pd
import scipy.stats as sts
import scikit_posthocs as sp
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv('EFIplus_medit.zip',compression='zip', sep=";")

In [4]:
df2 = df.dropna(how='all') # drops rows when all elements are missing values
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5011 entries, 0 to 5010
Columns: 164 entries, Site_code to Iberochondrostoma_sp
dtypes: float64(38), int64(120), object(6)
memory usage: 6.3+ MB


In [5]:
df2 = df.dropna() # drops rows when at least one element is a missing value
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
Index: 2822 entries, 18 to 5010
Columns: 164 entries, Site_code to Iberochondrostoma_sp
dtypes: float64(38), int64(120), object(6)
memory usage: 3.6+ MB


,Site_code,Latitude,Longitude,Country,Catchment_name,Galiza,Subsample,Calib_EFI_Medit,Calib_connect,Calib_hydrol,...,Squalius malacitanus,Squalius pyrenaicus,Squalius torgalensis,Thymallus thymallus,Tinca tinca,Zingel asper,Squalius sp,Barbatula sp,Phoxinus sp,Iberochondrostoma_sp
18,ES_03_0001,41.957268,-2.926857,Spain,Douro,0,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
19,ES_03_0002,41.850818,-2.591150,Spain,Douro,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
28,ES_03_0012,41.498026,-5.528753,Spain,Douro,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
32,ES_03_0016,41.586853,-3.072064,Spain,Douro,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
43,ES_03_0029,41.830177,-3.782420,Spain,Douro,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
import pandas as pd

# Crear la tabla de frecuencia cruzada
country_count = pd.crosstab(index=df2['Country'], columns='count')

# Ordenar el DataFrame por la columna 'count' en orden descendente
country_count_sorted = country_count.sort_values(by='count', ascending=False)

# Mostrar el DataFrame ordenado
print(country_count_sorted)

col_0     count
Country        
Spain      1905
Portugal    866
Italy        51


In [25]:
import pandas as pd
import scipy.stats as sts


data = pd.crosstab(df2['Country'], df2['Salmo trutta fario'])

data.head()

Salmo trutta fario,0,1
Country,,
Italy,26,25
Portugal,614,252
Spain,140,1765


In [27]:
# Ejecutar el test de chi-cuadrado
stat, p, df, expected_freq = sts.chi2_contingency(data)
print('Estadísticas=%.3f, p=%.6f' % (stat, p))

alpha = 0.05
if p > alpha:
    print('No se rechaza H0. Rechazar H0 tiene una probabilidad de error > 0.05')
else:
    print('Se rechaza H0 con una probabilidad de error < 0.05') # (si hay una diferencia entre los paises), no es independiente

Estadísticas=1216.393, p=0.000000
Se rechaza H0 con una probabilidad de error < 0.05


In [9]:

catchment_count = pd.crosstab(index = df2['Catchment_name'], columns='count')
catchment_count.head()

col_0,count
Catchment_name,
Algarve-Sotavento,6
Algarve-barlavento,10
Campania-South,5
Cantabrica,345
Catala,2


In [10]:
presence = df2[df2['Salmo trutta fario'] == 1]
absence = df2[df2['Salmo trutta fario'] == 0]
presence= presence['temp_ann'].dropna()
absence = absence['temp_ann'].dropna()

# Compute the test
stat, p = sts.kruskal(presence, absence)
print('F-statistics=%.3f, p=%.6f' % (stat, p))
alpha=0.05
if p > alpha:
 print('Fail to reject H0. Rejecting H0 has an error probability >0.05')
else:
 print('Reject H0 with an error probability <0.05)')

F-statistics=885.892, p=0.000000
Reject H0 with an error probability <0.05)


## Wine secction

In [11]:
df_vino = pd.read_csv('winequality_red.csv')
df_vino.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [14]:
bueno = df_vino.loc[df_vino['quality'] > 5, 'quality']
malo = df_vino.loc[df_vino['quality'] <= 5, 'quality']

In [16]:
# H0: The two samples are drawn from populations with equal distributions

stat, p = sts.ks_2samp(bueno,malo)
print('stat=%.3f, p-value=%.3f' % (stat, p))
alpha=0.05
if p > alpha:
 print('fail to reject H0. Rejecting H0 has an error probability >0.05')
else:
 print('reject H0 with an error probability <0.05)')

stat=1.000, p-value=0.000
reject H0 with an error probability <0.05)
